In [3]:
instructions = readlines("input") .|> x->split(x," ")

252-element Vector{Vector{SubString{String}}}:
 ["inp", "w"]
 ["mul", "x", "0"]
 ["add", "x", "z"]
 ["mod", "x", "26"]
 ["div", "z", "1"]
 ["add", "x", "12"]
 ["eql", "x", "w"]
 ["eql", "x", "0"]
 ["mul", "y", "0"]
 ["add", "y", "25"]
 ["mul", "y", "x"]
 ["add", "y", "1"]
 ["mul", "z", "y"]
 ⋮
 ["eql", "x", "w"]
 ["eql", "x", "0"]
 ["mul", "y", "0"]
 ["add", "y", "25"]
 ["mul", "y", "x"]
 ["add", "y", "1"]
 ["mul", "z", "y"]
 ["mul", "y", "0"]
 ["add", "y", "w"]
 ["add", "y", "14"]
 ["mul", "y", "x"]
 ["add", "z", "y"]

Many of the instructions look like no-ops (e.g. the second one, "mul x 0" does nothing as x is already initialized as 0. Anyway, my first approach is to write an interpreter that performs these operations.

In [16]:
initial_state() = Dict("w"=>0, "x" => 0, "y" => 0, "z" => 0)
initial_state()

Dict{String, Int64} with 4 entries:
  "w" => 0
  "x" => 0
  "z" => 0
  "y" => 0

In [17]:
possible_keys = keys(initial_state())

KeySet for a Dict{String, Int64} with 4 entries. Keys:
  "w"
  "x"
  "z"
  "y"

In [39]:
function perform_operation(state, operation, input)
    op3 = 0
    if length(operation)>2
        op3 = operation[3]
        if op3 ∈ possible_keys
            op3 = state[op3]
        else
            op3 = parse(Int, op3)
        end
    end
    if operation[1] == "inp"
        state[operation[2]] = pop!(input)
    elseif operation[1] == "add"
        state[operation[2]] += op3
    elseif operation[1] == "mul"
        state[operation[2]] *= op3
    elseif operation[1] == "div"
        state[operation[2]] ÷= op3 # integer division
    elseif operation[1] == "mod"
        state[operation[2]] %= op3
    elseif operation[1] == "eql"
        state[operation[2]] = state[operation[2]]==op3 ? 1 : 0
    else
        println("Illegal instruction: "*operation[1])
    end
end

perform_operation (generic function with 1 method)

In [32]:
function monad(input)
    state = initial_state()
    for inst in instructions
        perform_operation(state, inst, input)
    end
    return state
end

monad (generic function with 1 method)

In [33]:
monad([9,9,9,9,9,9,9,9,9,9,9,9,9,9])

Dict{String, Int64} with 4 entries:
  "w" => 9
  "x" => 1
  "z" => 5150970289
  "y" => 23

In [38]:
digits(9739234)

7-element Vector{Int64}:
 4
 3
 2
 9
 3
 7
 9

I don't expect this to work. But let's give brute-force a shot first.

In [73]:
for i in 99993999999999:-1:99993899999999
    d = digits(i)
    if 0 ∉ d
        s = monad(digits(i))
        if s["z"] == 0
            println(i)
            break
        end
    end
end

LoadError: InterruptException:

In [81]:
monad(digits(99993939993998))

Dict{String, Int64} with 4 entries:
  "w" => 8
  "x" => 0
  "z" => 11268
  "y" => 0

In [61]:
monad(digits(93999939939999))

Dict{String, Int64} with 4 entries:
  "w" => 9
  "x" => 1
  "z" => 195364361
  "y" => 23

In [54]:
monad(digits(99199199999999))

Dict{String, Int64} with 4 entries:
  "w" => 9
  "x" => 1
  "z" => 196436497
  "y" => 23

In [55]:
monad(digits(99999999999999))

Dict{String, Int64} with 4 entries:
  "w" => 9
  "x" => 1
  "z" => 5150970289
  "y" => 23

In [57]:
5150970289 / 198106217

26.001053207734515

Even with some reverse engineering this does not seem to work. So let's try another thing...
Try digit by digit and keep track of state. Only save distinct states and remember the highest number that leads there.

In [88]:
inp_positions = findall(x->x[1]=="inp", instructions)
push!(inp_positions, length(instructions)+1)
inp_positions'

1×15 adjoint(::Vector{Int64}) with eltype Int64:
 1  19  37  55  73  91  109  127  145  163  181  199  217  235  253

In [169]:
possible_states = Dict()
for i in 9:-1:7
    inp = [i]
    state = initial_state()
    for inst in instructions[inp_positions[1]:inp_positions[2]-1]
        perform_operation(state, inst, inp)
    end
    if state ∉ keys(possible_states)
        possible_states[state] = i
    end
end

In [170]:
possible_states

Dict{Any, Any} with 3 entries:
  Dict("w"=>7, "x"=>1, "z"=>14, "y"=>14) => 7
  Dict("w"=>8, "x"=>1, "z"=>15, "y"=>15) => 8
  Dict("w"=>9, "x"=>1, "z"=>16, "y"=>16) => 9

In [171]:
for j in 2:length(inp_positions)-1
    new_states = Dict()
    for oldstate in sort(collect(keys(possible_states)), by = x->possible_states[x], rev=true)
        for i in 9:-1:1
            inp = [i]
            state = copy(oldstate)
            for inst in instructions[inp_positions[j]:inp_positions[j+1]-1]
                perform_operation(state, inst, inp)
            end
            if state ∉ keys(new_states)
                new_states[state] = 10*possible_states[oldstate] + i
            end
        end
    end
    lowest_z = minimum([abs(x["z"]) for x in keys(new_states)])
    # heuristic! filter keys with high z values (unlikely to get down to 0)
    z_threshold = (lowest_z+1)*26*26*26*26
    possible_states = filter(x->abs(first(x)["z"])<z_threshold, new_states)
    println(length(possible_states))
    flush(stdout)
end

27
243
270
372
2565
2850
3610
5922
53298
22950
90234
91710
28166


In [172]:
filter(x->first(x)["z"]==0, possible_states)

Dict{Any, Any} with 5 entries:
  Dict("w"=>1, "x"=>0, "z"=>0, "y"=>0) => 79197919593981
  Dict("w"=>5, "x"=>0, "z"=>0, "y"=>0) => 79197919993985
  Dict("w"=>4, "x"=>0, "z"=>0, "y"=>0) => 79197919893984
  Dict("w"=>2, "x"=>0, "z"=>0, "y"=>0) => 79197919693982
  Dict("w"=>3, "x"=>0, "z"=>0, "y"=>0) => 79197919793983

In [174]:
sort(collect(values(filter(x->first(x)["z"]==0, possible_states))), rev=true)

5-element Vector{Any}:
 79197919993985
 79197919893984
 79197919793983
 79197919693982
 79197919593981

In [109]:
filter(x->first(x)["z"]==0, possible_states)

Dict{Any, Any} with 5 entries:
  Dict("w"=>1, "x"=>0, "z"=>0, "y"=>0) => 37195915593651
  Dict("w"=>5, "x"=>0, "z"=>0, "y"=>0) => 37195915971985
  Dict("w"=>4, "x"=>0, "z"=>0, "y"=>0) => 37195915893654
  Dict("w"=>2, "x"=>0, "z"=>0, "y"=>0) => 37195915693432
  Dict("w"=>3, "x"=>0, "z"=>0, "y"=>0) => 37195915793543

In [102]:
sort([x["z"] for x in keys(possible_states)])

920268-element Vector{Int64}:
         2
         2
         2
         3
         3
         3
         4
         4
         4
         5
         5
         5
         6
         ⋮
 198114213
 198114214
 198114215
 198114233
 198114234
 198114235
 198114236
 198114237
 198114238
 198114239
 198114240
 198114241

In [100]:
26*26

676

In [103]:
a = Dict(1=>'a',2=>'b',3=>'c')

Dict{Int64, Char} with 3 entries:
  2 => 'b'
  3 => 'c'
  1 => 'a'

In [104]:
filter(x->first(x)<3,a)

Dict{Int64, Char} with 2 entries:
  2 => 'b'
  1 => 'a'

## Check solutions found via approach above

In [133]:
monad(digits(76194919971875))

Dict{String, Int64} with 4 entries:
  "w" => 5
  "x" => 0
  "z" => 0
  "y" => 0

In [139]:
monad(digits(78196919971875))

Dict{String, Int64} with 4 entries:
  "w" => 5
  "x" => 0
  "z" => 0
  "y" => 0

In [175]:
monad(digits(79197919993985))

Dict{String, Int64} with 4 entries:
  "w" => 5
  "x" => 0
  "z" => 0
  "y" => 0

In [176]:
monad(digits(37195915593651))

Dict{String, Int64} with 4 entries:
  "w" => 1
  "x" => 0
  "z" => 0
  "y" => 0

In [181]:
monad(digits(13191913571211))

Dict{String, Int64} with 4 entries:
  "w" => 1
  "x" => 0
  "z" => 0
  "y" => 0

## Part II

In [177]:
possible_states = Dict()
for i in 1:3
    inp = [i]
    state = initial_state()
    for inst in instructions[inp_positions[1]:inp_positions[2]-1]
        perform_operation(state, inst, inp)
    end
    if state ∉ keys(possible_states)
        possible_states[state] = i
    end
end

In [178]:
possible_states

Dict{Any, Any} with 3 entries:
  Dict("w"=>2, "x"=>1, "z"=>9, "y"=>9)   => 2
  Dict("w"=>3, "x"=>1, "z"=>10, "y"=>10) => 3
  Dict("w"=>1, "x"=>1, "z"=>8, "y"=>8)   => 1

In [179]:
for j in 2:length(inp_positions)-1
    new_states = Dict()
    for oldstate in sort(collect(keys(possible_states)), by = x->possible_states[x])
        for i in 1:9
            inp = [i]
            state = copy(oldstate)
            for inst in instructions[inp_positions[j]:inp_positions[j+1]-1]
                perform_operation(state, inst, inp)
            end
            if state ∉ keys(new_states)
                new_states[state] = 10*possible_states[oldstate] + i
            end
        end
    end
    lowest_z = minimum([abs(x["z"]) for x in keys(new_states)])
    # heuristic! filter keys with high z values (unlikely to get down to 0)
    z_threshold = (lowest_z+1)*26*26*26*26
    possible_states = filter(x->abs(first(x)["z"])<z_threshold, new_states)
    println(length(possible_states))
    flush(stdout)
end

27
243
270
372
2565
2850
3612
5949
53541
23274
92502
95994
32324


In [180]:
sort(collect(values(filter(x->first(x)["z"]==0, possible_states))))

5-element Vector{Any}:
 13191913571211
 13191913671212
 13191913771213
 13191913871214
 13191913971215